In [ ]:
# Setup

# Install NLTK and other packages
!pip list | grep nltk
! pip install -U kaleido
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import nltk


nltk.download("punkt")
nltk.download("wordnet")

# Unzip per this stackoverflow: https://stackoverflow.com/questions/73849624/getting-error-while-submitting-notebook-on-kaggle-even-after-importing-nltk-libr
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
# Data cleaning and normalization

# Normalize/clean course data to the name, topic, link, text format for now

dataMit = pd.read_csv("/kaggle/input/dataset-of-1200-coursera-courses/MIT ocw.csv")
dataMit.columns = map(str.lower, dataMit.columns)
dataMit.rename(columns={"name ": "name"}, inplace=True)
dataMit.rename(columns={"course link": "link"}, inplace=True)
dataMit["text"] = dataMit["name"] + " " + dataMit["topic"]
dataMit["provider"] = "Massachussets Institute of Technology"
dataMit = dataMit[["name", "topic", "link", "provider", "text"]]


dataHarvard = pd.read_csv(
    "/kaggle/input/dataset-of-1200-coursera-courses/Harvard_university.csv"
)
dataHarvard.columns = map(str.lower, dataHarvard.columns)
dataHarvard.rename(columns={"link to course": "link", "about": "topic"}, inplace=True)
dataHarvard = dataHarvard[dataHarvard["price"] == "Free"]
dataHarvard["text"] = dataHarvard["name"] + " " + dataHarvard["topic"]
dataHarvard["provider"] = "Harvard University"
dataHarvard = dataHarvard[["name", "topic", "link", "provider", "text"]]


dataEdx = pd.read_csv("/kaggle/input/edx-courses-dataset-2021/EdX.csv")
dataEdx.columns = map(str.lower, dataEdx.columns)
dataEdx["topic"] = dataEdx["about"] + ". " + dataEdx["course description"]
dataEdx["provider"] = "edX - " + dataEdx["university"]
dataEdx["text"] = dataEdx["name"] + " " + dataEdx["topic"]
dataEdx = dataEdx[["name", "topic", "link", "provider", "text"]]


dataUdemy = pd.read_csv(
    "/kaggle/input/udemy-course-dataset-categories-ratings-and-trends/udemy_courses.csv"
)
dataUdemy.columns = map(str.lower, dataUdemy.columns)
dataUdemy.rename(
    columns={
        "title": "name",
        "headline": "topic",
        "url": "link",
    },
    inplace=True,
)
# only keep free courses
dataUdemy = dataUdemy[dataUdemy["is_paid"] == False]
# Since Udemy courses are user generated, filter only courses with rating over 4.5
dataUdemy["provider"] = "Udemy"
dataUdemy = dataUdemy[dataUdemy["rating"] > 4.5]
dataUdemy["text"] = dataUdemy["name"] + " " + dataUdemy["topic"]
dataUdemy = dataUdemy[["name", "topic", "link", "provider", "text"]]


dataCoursera = pd.read_csv("/kaggle/input/coursera-free-courses-dataset/coursera.csv")
dataCoursera.rename(
    columns={
        "title": "name",
        "skills": "topic",
        "url": "link",
    },
    inplace=True,
)
dataCoursera = dataCoursera[dataCoursera["price"] == "Free"]
dataCoursera["text"] = (
    dataCoursera["name"]
    + " "
    + np.where(pd.notna(dataCoursera["topic"]), dataCoursera["topic"], "")
)

dataCoursera["provider"] = "Coursera - " + dataCoursera["course_by"]
dataCoursera = dataCoursera[["name", "topic", "link", "provider", "text"]]

In [ ]:
def clean_text(text):
    lemma = WordNetLemmatizer()  # lemmatizer
    text = re.sub("[^A-Za-z0-9 ]", "", text)
    text = text.lower()
    tokens = word_tokenize(text)  # look into this tokenization
    tokens = [
        lemma.lemmatize(word)
        for word in tokens  # lemmatize words and remove stopwords
        if word not in stopwords.words("english")
    ]
    return " ".join(tokens)  # SBERT rrequires joined tokens


# Combine and clean data
data = pd.concat([dataUdemy, dataMit, dataHarvard, dataEdx, dataCoursera])
data["cleaned_text"] = data["text"].apply(
    clean_text
)  # Add clean text column to dataframe

# Drop non-english courses
indices_to_drop = [
    index
    for index, row in data.iterrows()
    if bool(
        re.search(
            r"[^\x00-\x7F\u2000-\u206F\u2600-\u26FF\u2700-\u27BF]", str(row["text"])
        )
    )
]
data = data.drop(indices_to_drop)

In [ ]:
# Initialize the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Get a list of the document embedding vector for each sentence in the cleaned text data. The indices will be aligned with the original course rows in dataframe
document_embeddings = model.encode(data["cleaned_text"].tolist())

In [ ]:
# Export a csv with embeddings for fastapi
data = data[["name", "topic", "link", "provider"]]
data.to_csv("courses.csv", index=False)
embeddings = pd.DataFrame(document_embeddings)
embeddings.to_csv("embeddings.csv", index=False)

In [ ]:
# Use previous functions to process user input into vector and use cosine
# Cosine Similarity to find the most related courses
# removed document_embeddings, data, model, top_n=5 as it was unnecessary abstraction layer


def recommend_courses(user_input: str, top_n=5) -> str:
    """Converts the user input to an embedding vector and compares it to list of courses embeddings,
    returning the 5 with the highest cosine similarity."""
    cleaned_input = clean_text(user_input)
    input_embedding = model.encode([cleaned_input])  # Model must be initialized
    similarities = cosine_similarity(input_embedding, document_embeddings)[0]
    top_indices = np.argsort(similarities)[-top_n:][::-1]
    recommendations = data.iloc[top_indices][["name", "topic", "link", "provider"]]
    return recommendations


user_input = "Python"
recommendations = recommend_courses(user_input)